In [54]:
import osmnx as ox
import pandas as pd
import numpy as np
from Data import Data 
import matplotlib.pyplot as plt
import nvector as nv
import osmapi as osm
import networkx as nx

In [55]:
df = Data('sample.csv').df
G = ox.graph_from_address('Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece')

In [56]:
def cross_track_dist(latA1,lonA1,latA2,lonA2,latB,lonB):
    frame = nv.FrameE(a=6371e3, f=0)
    
    pointA1 = frame.GeoPoint(latA1, lonA1, degrees=True)
    pointA2 = frame.GeoPoint(latA2, lonA2, degrees=True)
    pointB = frame.GeoPoint(latB, lonB, degrees=True)
    pathA = nv.GeoPath(pointA1, pointA2)

    s_xt = pathA.cross_track_distance(pointB, method='greatcircle')
    d_xt = pathA.cross_track_distance(pointB, method='euclidean')
    
    if(s_xt<0):
        s_xt=-1*s_xt
    
    val_txt = '{:4.2f}'.format(s_xt/1000)
    return val_txt

In [57]:
nodes = ox.distance.get_nearest_nodes(G, df['lat'], df['lon'], method='kdtree')
edges = ox.distance.get_nearest_edges(G, df['lat'], df['lon'], method='kdtree')
df['Nearest Node'] = nodes
edge_list_first = [edge[0] for edge in edges]
edge_list_second = [edge[1] for edge in edges]

df['Nearest Edge First NodeID'] =  edge_list_first
df['Nearest Edge Second NodeID'] =  edge_list_second
cross_track = []
for i in range(len(df)):
    lat1 = G.nodes[edge_list_first[i]]['y']
    lon1 = G.nodes[edge_list_first[i]]['x']
    lat2 = G.nodes[edge_list_second[i]]['y']
    lon2 = G.nodes[edge_list_second[i]]['x']
    latV = df['lat'].tolist()[i]
    lonV = df['lon'].tolist()[i]
    cross_track.append(cross_track_dist(lat1,lon1,lat2,lon2,latV,lonV))
    
df["Cross Track Distance"] = cross_track

In [58]:
def cross_dist(df):
    df['cross_dist_avg'] = \
        df.groupby('id', as_index=False, group_keys=False) \
        .apply(cross_dist_average)
    return df

In [79]:
def cross_dist_average(df):
    df2 = df["Cross Track Distance"].to_numpy()
    df = [float(i) for i in df2] 
    return df

In [80]:
cross_dist(df)

AssertionError: Gaps in blk ref_locs